In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import h5py
import glob

from SiPMStudio.analysis.dark import amp_dt, cross_talk_frac
from matplotlib.colors import LogNorm

from scipy.signal import find_peaks
from scipy.stats import linregress

In [ ]:
file_list = glob.glob("t2_v2/*room2*.h5")

In [ ]:
file_list

In [ ]:
test_file = h5py.File(file_list[5], "r")
dt = test_file["dt"][()]
timetags = test_file["timetag"][:]
blr_wf = test_file["/processed/channels/sipm/blr_wf"][:]
wf_len = test_file["/processed/channels/sipm/wf_len"][()]
adc_v = test_file["/processed/channels/sipm/adc_to_v"][()]
norm_charges = test_file["/processed/channels/sipm/norm_charge"][:]

In [ ]:
test_file.close()

In [ ]:
wf_time = np.arange(0, dt*wf_len, dt)

In [ ]:
def plot_waveforms(ax, time, waveforms, num=50, offset=0):
    for i in range(offset, num+offset):
        ax.plot(time, waveforms[i])
    ax.set_xlabel("Time (ns)")
    ax.set_ylabel("Amplitude (mV)")
    
def plot_charge(ax, charges, bins=1000, x_range=None):
    ax.hist(charges, bins=bins, range=x_range, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[0])
    ax.hist(charges, bins=bins, range=x_range, histtype="step", color=sns.color_palette()[0])
    ax.set_xlabel("P.E.")
    ax.set_ylabel("Counts")
    
def plot_dts(ax, dts, bins=1000, x_range=None):
    ax.hist(dts, bins=bins, range=x_range, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[0])
    n, bins, patches = ax.hist(dts, bins=bins, range=x_range, histtype="step", color=sns.color_palette()[0])
    ax.set_xlabel("Inter-times (ns)")
    ax.set_ylabel("Count")
    return n, bins
    
def plot_amp_dt(fig, ax, dts, amps, bins=[1000, 1000], x_range=[10, 5e3], y_range=[0, 600]):
    x_bins = np.logspace(np.log10(x_range[0]), np.log10(x_range[1]), bins[0])
    y_bins = np.linspace(y_range[0], y_range[1], bins[1])
    h, x, y, im = ax.hist2d(dts, amps, bins=[x_bins, y_bins], norm=LogNorm())
    ax.set_xlabel("Inter-times (ns)")
    ax.set_ylabel("Amplitude (mV)")
    ax.set_xscale("log")
    fig.colorbar(im, ax=ax)
    
def fit_exp(x, y):
    x_new = x[y > 0]
    y_new = y[y > 0]
    ln_y = np.log(y_new)
    slope, intercept, r, p, stderr = linregress(x_new, ln_y)
    scale = (1 / (1e-9*1e3))
    return abs(slope)*scale, stderr*scale, slope, intercept

In [ ]:
fig, ax = plt.subplots()
plot_waveforms(ax, wf_time, blr_wf*adc_v*1e3)

In [ ]:
fig, ax = plt.subplots()
plot_charge(ax, norm_charges[norm_charges > -0.12])
ax.set_yscale("log")

In [ ]:
wf_dts, wf_amps, wf_ids = amp_dt(timetags, blr_wf*adc_v*1e3, dt, height=100, distance=60)

In [ ]:
fig, ax = plt.subplots()
plot_amp_dt(fig, ax, wf_dts, wf_amps, bins=[200, 1000], x_range=[10, 1e4], y_range=[0, 400])

In [ ]:
fig, ax = plt.subplots()
n, bins = plot_dts(ax, wf_dts[wf_amps > 132], x_range=[0, 5e3], bins=150)
ax.set_yscale("log")

In [ ]:
bin_centers = (bins[1:] + bins[:-1]) / 2
cut = (bin_centers < 4500) & (bin_centers > 500)
rate, error, slope, intercept = fit_exp(bin_centers[cut], n[cut])

In [ ]:
inter_times = np.linspace(0, 5000, 100)
exp_func = np.exp(intercept + slope*inter_times)

In [ ]:
fig, ax = plt.subplots()
n, bins = plot_dts(ax, wf_dts, x_range=[0, 5e3], bins=150)
ax.plot(inter_times, exp_func, color="red", alpha=0.5, label=rf"{round(rate, 1)} kHz $\pm$ {round(error, 2)}")
ax.legend()
ax.set_yscale("log")

In [ ]:
cross_talk, cross_error = cross_talk_frac(norm_charges)

In [ ]:
print(cross_talk, cross_error)